<a href="https://colab.research.google.com/github/MaCroDmT/Automation_Engineering/blob/main/Test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx openpyxl


from docx import Document
import pandas as pd
import re
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

from google.colab import drive
drive.mount('/content/drive')

doc_path = "/content/drive/MyDrive/Sonia_Sweaters/Test_2/Style-6U128910-US.docx"
doc = Document(doc_path)

lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
for table in doc.tables:
    for row in table.rows:
        row_text = [cell.text.strip() for cell in row.cells if cell.text.strip()]
        if row_text:
            lines.append(" | ".join(row_text))


po_number, factory, country, port = "", "", "", ""
for line in lines:
    if "Purchase order #" in line:
        po_number = line.split("#")[-1].strip()
    if "Manufacturing Factory:" in line:
        factory = line.split(":")[-1].strip()
    if "Country of Origin:" in line:
        country = line.split(":")[-1].strip()
    if "Port:" in line:
        port = line.split(":")[-1].strip()


line_items = []
comments = []
terms_block = ""
in_terms = False

i = 0
while i < len(lines):
    line = lines[i]


    if line.startswith("Comment Type"):
        for j in range(i+1, len(lines)):
            if "|" in lines[j]:
                comments.append(lines[j])
            else:
                break
        i += len(comments)
        continue

    if line.startswith("This purchase order is subject to"):
        in_terms = True
    if in_terms:
        terms_block += " " + line.strip()

    if line.startswith("Line:") or line.startswith("Line: |"):
        item = {
            "PO Number": po_number,
            "Factory": factory,
            "Country": country,
            "Port": port,
            "Terms & Conditions": terms_block.strip()
        }

        context = " ".join(lines[i:i+7])
        fields = [f.strip() for f in context.split("|")]

        for j in range(0, len(fields)-1):
            key = fields[j].lower()
            val = fields[j+1].strip()

            if key == "line:":
                item["Line Number"] = val
            elif key == "style:":
                item["Style"] = val
            elif key == "season:":
                item["Season"] = val
            elif "current crd" in key:
                item["Current CRD"] = val
            elif "original crd" in key:
                item["Original CRD"] = val
            elif "logistics plan" in key:
                item["Logistics Plan"] = val
            elif key == "mode:":
                item["Mode"] = val
            elif "mode description" in key:
                item["Mode Description"] = val
            elif "ship to" in key:
                item["Ship To"] = val

        size_line = ""
        qty_line = ""
        for k in range(i+1, min(i+7, len(lines))):
            if re.search(r"\d+[M|T]\/", lines[k]):
                size_line = lines[k]
                if k+1 < len(lines):
                    qty_line = lines[k+1]
                break
        if size_line:
            item["Size Breakdown"] = size_line
        if qty_line:
            item["Qty Breakdown"] = qty_line


        for j in range(i, min(i+10, len(lines))):
            if "$" in lines[j]:
                try:
                    prices = re.findall(r"\$\s?[\d,]+\.\d+", lines[j])
                    if prices:
                        item["Unit Price (USD)"] = float(prices[0].replace("$", "").replace(",", ""))
                        item["Total USD"] = float(prices[-1].replace("$", "").replace(",", ""))
                        item["Total Units"] = int(lines[j+1].split("|")[0].strip())
                        break
                except:
                    pass

        line_items.append(item)
        i += 1
    else:
        i += 1


df = pd.DataFrame(line_items)


comments_df = pd.DataFrame(comments, columns=["Comment"])

output_path = "/content/drive/MyDrive/Sonia_Sweaters/Test_2/W25_2_Tracking_Chart.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="Line Items")
    comments_df.to_excel(writer, index=False, sheet_name="Comments")

print("Excel_file_created_at:", output_path)


def style_excel_file(path):
    wb = openpyxl.load_workbook(path)

    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )

    header_fill = PatternFill(start_color="BDD7EE", end_color="BDD7EE", fill_type="solid")
    alt_fill = PatternFill(start_color="F3F3F3", end_color="F3F3F3", fill_type="solid")

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]


        for col in ws.columns:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
            adjusted_width = max_length + 2
            ws.column_dimensions[column].width = adjusted_width


        for cell in ws[1]:
            cell.font = Font(bold=True, color="000000")
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.border = thin_border


        for row_idx, row in enumerate(ws.iter_rows(min_row=2), start=2):
            for cell in row:
                cell.alignment = Alignment(vertical="top", wrap_text=True)
                cell.border = thin_border
                if row_idx % 2 == 0:
                    cell.fill = alt_fill


    wb.save(path)

style_excel_file(output_path)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Excel_file_created_at: /content/drive/MyDrive/Sonia_Sweaters/Test_2/W25_2_Tracking_Chart.xlsx
